# STEP 0
* In the Neo4j desktop create a server/databse instance and in the server create a database.
* Put all the csv files in adventureworks_data.zip in the Neo4j import folder.
* Put this ipynb file in the import folder as well.


# STEP 1
* The block help improt all the csv files into Neo4j after putting data files in Neo4j import folder. This step crete Lables(tables) and Nodes(rows in tables).

In [ ]:
import os
import pandas as pd
from neo4j import GraphDatabase

# Neo4j Connection
neo4j_uri = "bolt://localhost:7687"
neo4j_user = "neo4j"
neo4j_password = "" # please remember to change this
node_cmd = open('node_cmd.txt','w+')
driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_user, neo4j_password))

# Function to determine the Neo4j data type based on Pandas' dtypes
def get_neo4j_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return "toFloat"
    elif pd.api.types.is_float_dtype(dtype):
        return "toFloat"
    elif pd.api.types.is_bool_dtype(dtype):
        return "toBoolean"
    else:
        return ""  # Default to string (Neo4j stores it as-is)

# Load Data into Neo4j
with driver.session() as session:
    # put the filepath of the import folder
    for subdir, dirs, files in os.walk("C:/Users/shuyu/Neo4jDesktop/relate-data/dbmss/dbms-d14f5678-d6e5-4615-9a66-f1ab451ad7d3/import"):
        for file in files:
            filepath = os.path.join(subdir, file)

            if filepath.endswith(".csv"):
                df = pd.read_csv(filepath)  # Read CSV and infer data types
                filename = file
                tablename = file.split('.csv')[0].replace('.', '')  # Extract table name
                
                cols = "{"
                for col in df.columns:
                    dtype = df[col].dtype  # Get detected dtype from Pandas
                    conversion = get_neo4j_type(dtype)
                    if conversion:
                        cols += f"{col}: {conversion}(row.{col}),"
                    else:
                        cols += f"{col}: row.{col},"  # String case
                
                cols = cols.rstrip(",") + "}"  # Remove trailing comma
                
                # Generate Cypher command dynamically, change to USE your database name
                cmd = f"USE adventure\nLOAD CSV WITH HEADERS FROM 'file:///{filename}' AS row\nCREATE (:{tablename} {cols});\n"
                node_cmd.write(cmd)
                session.run(cmd)  # Execute the Cypher query

print("CSV Data Imported into Neo4j with Correct Data Types!")

# Close Connection
node_cmd.close()
driver.close()


CSV Data Imported into Neo4j with Correct Data Types!


# STEP 2
* Create all the indexes and relationships.
* Run the code in the Neo4j desktop.  
## Create Index
CREATE INDEX HumanResourcesDepartmentIndex FOR (d:HumanResourcesDepartment) ON (d.DepartmentID);  
CREATE INDEX PersonAddressIndex FOR (a:PersonAddress) ON (a.AddressID);  
CREATE INDEX PersonBusinessEntityIndex FOR (b:PersonBusinessEntity) ON (b.BusinessEntityID);  
CREATE INDEX ProductionProductIndex FOR (p:ProductionProduct) ON (p.ProductID);  
CREATE INDEX ProductionProductCategoryIndex FOR (pc:ProductionProductCategory) ON (pc.ProductCategoryID);  
CREATE INDEX ProductionProductSubcategoryIndex FOR (psubc:ProductionProductSubcategory) ON (psubc.ProductSubcategoryID);  
CREATE INDEX ProductionTransactionHistoryIndex FOR (t:ProductionTransactionHistory) ON (t.TransactionID);  
CREATE INDEX ProductionWorkOrderIndex FOR (wo:ProductionWorkOrder) ON (wo.WorkOrderID);  
CREATE INDEX SalesCustomerIndex FOR (c:SalesCustomer) ON (c.CustomerID);  
CREATE INDEX SalesSalesOrderDetailIndex FOR (od:SalesFilteredSalesOrderDetail) ON (od.SalesOrderDetailID);  
CREATE INDEX SalesSalesOrderHeaderWithDurationIndex FOR (oh:SalesFilteredSalesOrderHeaderWithDuration) ON (oh.SalesOrderID);  
CREATE INDEX SalesSalesOrderHeaderIndex FOR (oh:SalesFilteredSalesOrderHeader) ON (oh.SalesOrderID);  

## Create relationship
MATCH (c:SalesCustomer)  
MATCH (oh:SalesFilteredSalesOrderHeader)  
WHERE c.CustomerID = oh.CustomerID
MERGE (c)-[:PLACED_ORDER]->(oh);  

MATCH (c:SalesCustomer)  
MATCH (oht:SalesFilteredSalesOrderHeaderWithDuration)  
WHERE c.CustomerID = oht.CustomerID  
MERGE (c)-[:PLACED_ORDER]->(oht);  

MATCH (p:ProductionProduct)  
MATCH (k:ProductionProductKeywords)  
WHERE p.ProductID = k.ProductID  
MERGE (p)-[:HAS_KEYWORD]->(k);  

MATCH (p:ProductionProduct)  
MATCH (subc:ProductionProductSubcategory)  
WHERE p.ProductSubcategoryID = subc.ProductSubcategoryID  
MERGE (p)-[:PART_OF_SUBCAT]->(subc);  

MATCH (subc:ProductionProductSubcategory)  
MATCH (cat:ProductionProductCategory)  
WHERE cat.ProductCategoryID = subc.ProductCategoryID  
MERGE (subc)-[:BELONGS_TO]->(cat);  

MATCH (od:SalesFilteredSalesOrderDetail)  
MATCH (p: ProductionProduct)  
WHERE od.ProductID = p.ProductID  
MERGE (od)-[:HAS_PRODUCT]->(p);  

MATCH (oh:SalesFilteredSalesOrderHeader)  
MATCH (od: SalesFilteredSalesOrderDetail)  
WHERE oh.SalesOrderID = od.SalesOrderID  
MERGE (oh)-[:HAS_Detail]->(od);  

MATCH (oht:SalesFilteredSalesOrderHeaderWithDuration)  
MATCH (od: SalesFilteredSalesOrderDetail)  
WHERE oht.SalesOrderID = od.SalesOrderID  
MERGE (oht)-[:HAS_Detail]->(od);  

MATCH (p:ProductionProduct)  
MATCH (k:ProductionProductKeywords)  
WHERE p.ProductID = k.ProductID  
MERGE (p)-[:HAS_KEYWORD]->(k);

# STEP 3
* Run the cypher query either in Neo4j desktop or in the ipynb file

## Recommender (only given customer ID)

If want to run in the Neo4j desktop. Set the CustomerID manually in Neo4j by running :param customer_id => 29715.0

In [ ]:
from neo4j import GraphDatabase
import pandas as pd

# Neo4j Connection Settings
neo4j_uri = "bolt://localhost:7687"
neo4j_user = "neo4j"
neo4j_password = "" # please remember to change this

# Connect to Neo4j
driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_user, neo4j_password))

def get_product_recommendations(customer_id):
    """
    Given a customer ID, this function retrieves the top 10 recommended products from Neo4j.
    The recommendation is based on collaborative filtering and purchase frequency.
    It also ensures that no duplicate product IDs are returned.
    """
    cypher_query = """
   // Step 1: Find the products purchased by the target customer 
    MATCH (targetCustomer:SalesCustomer {CustomerID: $customer_id})-[:PLACED_ORDER]->(order:SalesFilteredSalesOrderHeaderWithDuration) 
    MATCH (order)-[:HAS_Detail]->(:SalesFilteredSalesOrderDetail)-[:HAS_PRODUCT]->(purchased:ProductionProduct)
    WITH targetCustomer, COLLECT(DISTINCT purchased.ProductID) AS targetProducts
    
    // Step 2: Find other customers who bought similar products
    MATCH (similarCustomer:SalesCustomer)-[:PLACED_ORDER]->(similarOrder:SalesFilteredSalesOrderHeaderWithDuration)
    WHERE similarCustomer <> targetCustomer   
    MATCH (similarOrder)-[:HAS_Detail]->(:SalesFilteredSalesOrderDetail)-[:HAS_PRODUCT]->(sharedProduct:ProductionProduct)
    WHERE sharedProduct.ProductID IN targetProducts
    WITH targetCustomer, similarCustomer, COLLECT(DISTINCT sharedProduct.ProductID) AS sharedProducts, targetProducts
    
    // Calculate similarity based on the overlap of purchased products
    WITH targetCustomer, similarCustomer, sharedProducts, targetProducts,
         size(sharedProducts) AS sharedProductCount,
         size(targetProducts) AS targetProductCount,
         COALESCE((size(sharedProducts) * 1.0 / NULLIF(size(targetProducts), 0)), 0) AS similarityScore  // NULLIF avoids division by zero
    
    // Step 3: Filter out products already purchased by the target customer
    MATCH (similarCustomer)-[:PLACED_ORDER]->(:SalesFilteredSalesOrderHeader)
          -[:HAS_Detail]->(:SalesFilteredSalesOrderDetail)-[:HAS_PRODUCT]->(recommended:ProductionProduct)
    WHERE NOT recommended.ProductID IN targetProducts
    
    // Step 4: Optional match for product categories and subcategories
    OPTIONAL MATCH (recommended)-[:PART_OF_SUBCAT]->(subCategory:ProductionProductSubcategory)
    OPTIONAL MATCH (subCategory)-[:BELONGS_TO]->(category:ProductionProductCategory)
    
    // Step 5: Ensure all nullable values are handled
    WITH DISTINCT recommended.ProductID AS ProductID, recommended.Name AS ProductName, 
        COALESCE(category.ProductCategoryID, -1) AS CategoryID, COALESCE(category.Name, 'Unknown') AS CategoryName, 
        COALESCE(subCategory.ProductSubcategoryID, -1) AS SubcategoryID, COALESCE(subCategory.Name, 'Unknown') AS SubcategoryName, 
        COALESCE(COUNT(DISTINCT similarCustomer), 0) AS collaborativeScore, 
        COALESCE(COUNT(DISTINCT subCategory), 0) AS categoryMatchScore, recommended, targetProducts
    
    // Step 6: Compute keyword overlap count as keywordSimilarityScore 
    OPTIONAL MATCH (recommended)-[:HAS_KEYWORD]->(rk:ProductionProductKeywords) 
    OPTIONAL MATCH (targetProduct:ProductionProduct)-[:HAS_KEYWORD]->(pk:ProductionProductKeywords)
    WHERE targetProduct.ProductID IN targetProducts AND rk.Keyword = pk.Keyword
    WITH ProductID, ProductName, CategoryID, CategoryName, SubcategoryID, SubcategoryName, 
         collaborativeScore, 
         COALESCE(categoryMatchScore, 0) AS categoryMatchScore, 
         COALESCE(COUNT(DISTINCT rk.Keyword), 0) AS keywordSimilarityScore
    
    // Step 7: Return top recommended products sorted by a combination of collaborative score and category match
    RETURN ProductID, ProductName, CategoryID, CategoryName, SubcategoryID, SubcategoryName, 
           collaborativeScore, categoryMatchScore, keywordSimilarityScore,
        (collaborativeScore * 2 + categoryMatchScore + keywordSimilarityScore) AS RecommendationScore
    ORDER BY RecommendationScore DESC, collaborativeScore DESC
    LIMIT 10;

    """

    # Execute query
    with driver.session() as session:
        result = session.run(cypher_query, customer_id=customer_id)
        data = result.data()

    # Convert to Pandas DataFrame
    df = pd.DataFrame(data)

    return df

# Example usage in Jupyter Notebook
customer_id = 29715.0  # Replace with the customer ID you want
recommendations = get_product_recommendations(customer_id)
print(recommendations)  # Display result


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.AggregationSkippedNull} {category: UNRECOGNIZED} {title: The query contains an aggregation function that skips null values.} {description: null value eliminated in set function.} {position: None} for query: "\n   // Step 1: Find the products purchased by the target customer \n    MATCH (targetCustomer:SalesCustomer {CustomerID: $customer_id})-[:PLACED_ORDER]->(order:SalesFilteredSalesOrderHeaderWithDuration) \n    MATCH (order)-[:HAS_Detail]->(:SalesFilteredSalesOrderDetail)-[:HAS_PRODUCT]->(purchased:ProductionProduct)\n    WITH targetCustomer, COLLECT(DISTINCT purchased.ProductID) AS targetProducts\n    \n    // Step 2: Find other customers who bought similar products\n    MATCH (similarCustomer:SalesCustomer)-[:PLACED_ORDER]->(similarOrder:SalesFilteredSalesOrderHeaderWithDuration)\n    WHERE similarCustomer <> targetCustomer   \n    MATCH (similarOrder)-[:HAS_Detail]->(:SalesFiltered

Neo4j Warnings Suppressed: 1
   ProductID                      ProductName  CategoryID CategoryName  \
0      715.0       Long-Sleeve Logo Jersey, L         3.0     Clothing   
1      714.0       Long-Sleeve Logo Jersey, M         3.0     Clothing   
2      859.0            Half-Finger Gloves, M         3.0     Clothing   
3      870.0            Water Bottle - 30 oz.         4.0  Accessories   
4      877.0            Bike Wash - Dissolver         4.0  Accessories   
5      884.0  Short-Sleeve Classic Jersey, XL         3.0     Clothing   
6      864.0                  Classic Vest, S         3.0     Clothing   
7      856.0              Men's Bib-Shorts, M         3.0     Clothing   
8      876.0              Hitch Rack - 4-Bike         4.0  Accessories   
9      854.0                Women's Tights, L         3.0     Clothing   

   SubcategoryID    SubcategoryName  collaborativeScore  categoryMatchScore  \
0           21.0            Jerseys                 801                   3  

## Recommender (given customer ID & category name)

If want to run in the Neo4j desktop. Set the CustomerID manually in Neo4j by running :param category_name => 'Clothing'

In [ ]:
from neo4j import GraphDatabase
import pandas as pd



# Neo4j Connection Settings
neo4j_uri = "bolt://localhost:7687"
neo4j_user = "neo4j"
neo4j_password = "" # please remember to change this

# Connect to Neo4j
driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_user, neo4j_password))

def get_product_recommendations(customer_id, category_name):
    """
    Given a customer ID and a CategoryName, this function retrieves the top 10 recommended products from Neo4j.
    The recommendation is based on collaborative filtering, purchase frequency, and category filtering.
    It also ensures that no duplicate product IDs are returned.
    """
    cypher_query = """
    // Step 1: Find the products purchased by the target customer 
    MATCH (targetCustomer:SalesCustomer {CustomerID: $customer_id})-[:PLACED_ORDER]->(order:SalesFilteredSalesOrderHeaderWithDuration) 
    MATCH (order)-[:HAS_Detail]->(:SalesFilteredSalesOrderDetail)-[:HAS_PRODUCT]->(purchased:ProductionProduct)
    WITH targetCustomer, COLLECT(DISTINCT purchased.ProductID) AS targetProducts
    
    // Step 2: Find other customers who bought similar products
    MATCH (similarCustomer:SalesCustomer)-[:PLACED_ORDER]->(similarOrder:SalesFilteredSalesOrderHeaderWithDuration)
    WHERE similarCustomer <> targetCustomer   
    MATCH (similarOrder)-[:HAS_Detail]->(:SalesFilteredSalesOrderDetail)-[:HAS_PRODUCT]->(sharedProduct:ProductionProduct)
    WHERE sharedProduct.ProductID IN targetProducts
    WITH targetCustomer, similarCustomer, COLLECT(DISTINCT sharedProduct.ProductID) AS sharedProducts, targetProducts
    
    // Calculate similarity based on the overlap of purchased products
    WITH targetCustomer, similarCustomer, sharedProducts, targetProducts,
         size(sharedProducts) AS sharedProductCount,
         size(targetProducts) AS targetProductCount,
         COALESCE((size(sharedProducts) * 1.0 / NULLIF(size(targetProducts), 0)), 0) AS similarityScore  // NULLIF avoids division by zero
    
    // Step 3: Filter out products already purchased by the target customer
    MATCH (similarCustomer)-[:PLACED_ORDER]->(:SalesFilteredSalesOrderHeader)
          -[:HAS_Detail]->(:SalesFilteredSalesOrderDetail)-[:HAS_PRODUCT]->(recommended:ProductionProduct)
    WHERE NOT recommended.ProductID IN targetProducts
    
    // Step 4: Optional match for product categories and subcategories
    OPTIONAL MATCH (recommended)-[:PART_OF_SUBCAT]->(subCategory:ProductionProductSubcategory)
    OPTIONAL MATCH (subCategory)-[:BELONGS_TO]->(category:ProductionProductCategory)
    
    // Step 5: Ensure all nullable values are handled
    WITH DISTINCT recommended.ProductID AS ProductID, recommended.Name AS ProductName, 
        COALESCE(category.ProductCategoryID, -1) AS CategoryID, COALESCE(category.Name, 'Unknown') AS CategoryName, 
        COALESCE(subCategory.ProductSubcategoryID, -1) AS SubcategoryID, COALESCE(subCategory.Name, 'Unknown') AS SubcategoryName, 
        COALESCE(COUNT(DISTINCT similarCustomer), 0) AS collaborativeScore, 
        COALESCE(COUNT(DISTINCT subCategory), 0) AS categoryMatchScore, recommended, targetProducts
    
    // Step 6: Compute keyword overlap count as keywordSimilarityScore 
    OPTIONAL MATCH (recommended)-[:HAS_KEYWORD]->(rk:ProductionProductKeywords) 
    OPTIONAL MATCH (targetProduct:ProductionProduct)-[:HAS_KEYWORD]->(pk:ProductionProductKeywords)
    WHERE targetProduct.ProductID IN targetProducts AND rk.Keyword = pk.Keyword
    WITH ProductID, ProductName, CategoryID, CategoryName, SubcategoryID, SubcategoryName, 
         collaborativeScore, 
         COALESCE(categoryMatchScore, 0) AS categoryMatchScore, 
         COALESCE(COUNT(DISTINCT rk.Keyword), 0) AS keywordSimilarityScore
    
    // Step 7: If CategoryName is provided, filter by CategoryName
    WHERE ($category_name IS NULL OR CategoryName = $category_name)
    
    // Step 8: Return top recommended products sorted by scores
    RETURN ProductID, ProductName, CategoryID, CategoryName, SubcategoryID, SubcategoryName, 
        collaborativeScore, categoryMatchScore, keywordSimilarityScore,
        (collaborativeScore * 2 + categoryMatchScore + keywordSimilarityScore) AS RecommendationScore
    ORDER BY RecommendationScore DESC, collaborativeScore DESC, keywordSimilarityScore DESC
    LIMIT 10;
    """

    # Execute query
    with driver.session() as session:
        result = session.run(cypher_query, customer_id=customer_id, category_name=category_name)
        data = result.data()

    # Convert to Pandas DataFrame
    df = pd.DataFrame(data)

    return df

# Example usage in Jupyter Notebook
customer_id = 29715.0  # Replace with the customer ID you want
category_name = 'Clothing'  # Replace with the desired category name
recommendations = get_product_recommendations(customer_id, category_name)
print(recommendations)  # Display result


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.AggregationSkippedNull} {category: UNRECOGNIZED} {title: The query contains an aggregation function that skips null values.} {description: null value eliminated in set function.} {position: None} for query: "\n    // Step 1: Find the products purchased by the target customer \n    MATCH (targetCustomer:SalesCustomer {CustomerID: $customer_id})-[:PLACED_ORDER]->(order:SalesFilteredSalesOrderHeaderWithDuration) \n    MATCH (order)-[:HAS_Detail]->(:SalesFilteredSalesOrderDetail)-[:HAS_PRODUCT]->(purchased:ProductionProduct)\n    WITH targetCustomer, COLLECT(DISTINCT purchased.ProductID) AS targetProducts\n    \n    // Step 2: Find other customers who bought similar products\n    MATCH (similarCustomer:SalesCustomer)-[:PLACED_ORDER]->(similarOrder:SalesFilteredSalesOrderHeaderWithDuration)\n    WHERE similarCustomer <> targetCustomer   \n    MATCH (similarOrder)-[:HAS_Detail]->(:SalesFiltere

   ProductID                      ProductName  CategoryID CategoryName  \
0      715.0       Long-Sleeve Logo Jersey, L         3.0     Clothing   
1      714.0       Long-Sleeve Logo Jersey, M         3.0     Clothing   
2      859.0            Half-Finger Gloves, M         3.0     Clothing   
3      884.0  Short-Sleeve Classic Jersey, XL         3.0     Clothing   
4      864.0                  Classic Vest, S         3.0     Clothing   
5      856.0              Men's Bib-Shorts, M         3.0     Clothing   
6      854.0                Women's Tights, L         3.0     Clothing   
7      881.0   Short-Sleeve Classic Jersey, S         3.0     Clothing   
8      849.0           Men's Sports Shorts, M         3.0     Clothing   
9      861.0            Full-Finger Gloves, S         3.0     Clothing   

   SubcategoryID SubcategoryName  collaborativeScore  categoryMatchScore  \
0           21.0         Jerseys                 801                   3   
1           21.0         Jerseys 

## Recommender (given customer ID, category name & max duration)
* If want to run in the Neo4j desktop. Set the CustomerID manually in Neo4j by running :param max_duration => 9

In [74]:
from neo4j import GraphDatabase
import pandas as pd


# Neo4j Connection Settings
neo4j_uri = "bolt://localhost:7687"
neo4j_user = "neo4j"
neo4j_password = "" # please remember to change this

# Connect to Neo4j
driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_user, neo4j_password))

def get_product_recommendations(customer_id, category_name, max_duration):
    """
    Given a customer ID, a CategoryName, and a max duration, this function retrieves the top 10 recommended products from Neo4j.
    The recommendation is based on collaborative filtering, purchase frequency, and category filtering.
    It ensures that only orders with a duration <= max_duration are considered.
    """
    cypher_query = """
    // Step 1: Find the products purchased by the target customer within the given duration limit
    MATCH (targetCustomer:SalesCustomer {CustomerID: $customer_id})-[:PLACED_ORDER]->(order:SalesFilteredSalesOrderHeaderWithDuration)
    WHERE order.Duration <= $max_duration  
    MATCH (order)-[:HAS_Detail]->(:SalesFilteredSalesOrderDetail)-[:HAS_PRODUCT]->(purchased:ProductionProduct)
    WITH targetCustomer, COLLECT(DISTINCT purchased.ProductID) AS targetProducts
    
    // Step 2: Find other customers who bought similar products
    MATCH (similarCustomer:SalesCustomer)-[:PLACED_ORDER]->(similarOrder:SalesFilteredSalesOrderHeaderWithDuration)
    WHERE similarCustomer <> targetCustomer AND similarOrder.Duration <= $max_duration  
    MATCH (similarOrder)-[:HAS_Detail]->(:SalesFilteredSalesOrderDetail)-[:HAS_PRODUCT]->(sharedProduct:ProductionProduct)
    WHERE sharedProduct.ProductID IN targetProducts
    WITH targetCustomer, similarCustomer, COLLECT(DISTINCT sharedProduct.ProductID) AS sharedProducts, targetProducts

// Calculate similarity based on the overlap of purchased products
    WITH targetCustomer, similarCustomer, sharedProducts, targetProducts,
        size(sharedProducts) AS sharedProductCount,
        size(targetProducts) AS targetProductCount,
        (size(sharedProducts) * 1.0 / size(targetProducts)) AS similarityScore
    
    // Step 3: Find products not yet purchased by the target customer
    MATCH (similarCustomer)-[:PLACED_ORDER]->(similarOrder)-[:HAS_Detail]->(:SalesFilteredSalesOrderDetail)-[:HAS_PRODUCT]->(recommended:ProductionProduct)
    WHERE NOT recommended.ProductID IN targetProducts
    
    // Step 4: Get product category details
    OPTIONAL MATCH (recommended)-[:PART_OF_SUBCAT]->(subCategory:ProductionProductSubcategory)
    OPTIONAL MATCH (subCategory)-[:BELONGS_TO]->(category:ProductionProductCategory)
    
    // Step 5: Calculate recommendation score
    WITH DISTINCT recommended.ProductID AS ProductID, recommended.Name AS ProductName, 
        category.ProductCategoryID AS CategoryID, category.Name AS CategoryName, 
        subCategory.ProductSubcategoryID AS SubcategoryID, subCategory.Name AS SubcategoryName, 
        COALESCE(COUNT(DISTINCT similarCustomer), 0) AS collaborativeScore, 
        COALESCE(COUNT(DISTINCT subCategory), 0) AS categoryMatchScore, recommended, targetProducts

    // Step 6: Compute keyword overlap count as keywordSimilarityScore 
    OPTIONAL MATCH (recommended)-[:HAS_KEYWORD]->(rk:ProductionProductKeywords) 
    OPTIONAL MATCH (targetProduct:ProductionProduct)-[:HAS_KEYWORD]->(pk:ProductionProductKeywords)
    WHERE targetProduct.ProductID IN targetProducts AND rk.Keyword = pk.Keyword
    WITH ProductID, ProductName, CategoryID, CategoryName, SubcategoryID, SubcategoryName, 
         collaborativeScore, 
         COALESCE(categoryMatchScore, 0) AS categoryMatchScore, 
         COALESCE(COUNT(DISTINCT rk.Keyword), 0) AS keywordSimilarityScore
    
    // Step 7: If CategoryName is provided, filter by CategoryName
    WHERE ($category_name IS NULL OR CategoryName = $category_name)
    
    // Step 8: Return top recommended products sorted by scores
    RETURN ProductID, ProductName, CategoryID, CategoryName, SubcategoryID, SubcategoryName, 
        collaborativeScore, categoryMatchScore, keywordSimilarityScore,
        (collaborativeScore * 2 + categoryMatchScore + keywordSimilarityScore) AS RecommendationScore
    ORDER BY RecommendationScore DESC, collaborativeScore DESC, keywordSimilarityScore DESC
    LIMIT 10;
    """

    # Execute query
    with driver.session() as session:
        result = session.run(cypher_query, customer_id=customer_id, category_name=category_name, max_duration=max_duration)
        data = result.data()

    # Convert to Pandas DataFrame
    df = pd.DataFrame(data)

    return df

# Example usage
customer_id = 29715.0  # Replace with the customer ID you want
category_name = 'Clothing'  # Replace with the desired category name
max_duration = 9  # Replace with the maximum allowed duration (e.g., 5 days)
recommendations = get_product_recommendations(customer_id, category_name, max_duration)
print(recommendations)  # Display result

   ProductID                      ProductName  CategoryID CategoryName  SubcategoryID SubcategoryName  collaborativeScore  categoryMatchScore  keywordSimilarityScore  RecommendationScore
0      715.0       Long-Sleeve Logo Jersey, L         3.0     Clothing           21.0         Jerseys                1068                   3                       2                 2141
1      714.0       Long-Sleeve Logo Jersey, M         3.0     Clothing           21.0         Jerseys                 808                   3                       3                 1622
2      859.0            Half-Finger Gloves, M         3.0     Clothing           20.0          Gloves                 748                   3                       4                 1503
3      884.0  Short-Sleeve Classic Jersey, XL         3.0     Clothing           21.0         Jerseys                 628                   3                       3                 1262
4      864.0                  Classic Vest, S         3.0     Clo

# Final output with:
customer_id = 29715.0  
category_name = 'Clothing'  
max_duration = 9  

In [75]:
recommendations[['ProductID', 'ProductName', 'CategoryName', 'SubcategoryName', 
        'collaborativeScore', 'categoryMatchScore', 'keywordSimilarityScore',
        'RecommendationScore']]

,ProductID,ProductName,CategoryName,SubcategoryName,collaborativeScore,categoryMatchScore,keywordSimilarityScore,RecommendationScore
0,715.0,"Long-Sleeve Logo Jersey, L",Clothing,Jerseys,1068,3,2,2141
1,714.0,"Long-Sleeve Logo Jersey, M",Clothing,Jerseys,808,3,3,1622
2,859.0,"Half-Finger Gloves, M",Clothing,Gloves,748,3,4,1503
3,884.0,"Short-Sleeve Classic Jersey, XL",Clothing,Jerseys,628,3,3,1262
4,864.0,"Classic Vest, S",Clothing,Vests,616,3,3,1238
5,856.0,"Men's Bib-Shorts, M",Clothing,Bib-Shorts,604,3,4,1215
6,854.0,"Women's Tights, L",Clothing,Tights,576,3,3,1158
7,881.0,"Short-Sleeve Classic Jersey, S",Clothing,Jerseys,500,3,2,1005
8,849.0,"Men's Sports Shorts, M",Clothing,Shorts,384,3,3,774
9,861.0,"Full-Finger Gloves, S",Clothing,Gloves,352,3,4,711
